In [59]:
import pandas as pd
df = pd.read_csv('ATP.csv')

/Users/petersinka/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3,5,6,7,8,9,12,14,15,16,17,18,19,22,24,25,26,28,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [60]:
df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
0,1968-580,Australian Chps.,Grass,64,G,19680119,1,110023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1968-580,Australian Chps.,Grass,64,G,19680119,2,109803,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1968-580,Australian Chps.,Grass,64,G,19680119,3,100257,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1968-580,Australian Chps.,Grass,64,G,19680119,4,100105,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1968-580,Australian Chps.,Grass,64,G,19680119,5,109966,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
df[df['tourney_date'].str.len() > 8]

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
160636,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced


In [71]:
len(df)

163966

In [61]:
# Most majors
gs_winner_df = df[(df['tourney_level'] == 'G') & (df['round'] == 'F')]
grouped = gs_winner_df.groupby('winner_name').count().sort_values('winner_id', ascending= False).ix[:,0]
grouped

winner_name
Roger Federer            18
Rafael Nadal             14
Pete Sampras             14
Novak Djokovic           12
Bjorn Borg               11
Ivan Lendl                8
Andre Agassi              8
Jimmy Connors             8
Mats Wilander             7
John Mcenroe              7
Stefan Edberg             6
Boris Becker              6
John Newcombe             5
Rod Laver                 5
Jim Courier               4
Ken Rosewall              4
Guillermo Vilas           4
Gustavo Kuerten           3
Jan Kodes                 3
Arthur Ashe               3
Andy Murray               3
Stanislas Wawrinka        3
Patrick Rafter            2
Lleyton Hewitt            2
Sergi Bruguera            2
Stan Smith                2
Marat Safin               2
Yevgeny Kafelnikov        2
Johan Kriek               2
Ilie Nastase              2
Albert Costa              1
Richard Krajicek          1
Yannick Noah              1
Thomas Muster             1
Thomas Johansson          1
Andres G

In [62]:
# List of all players in the data
winner_names = df[['winner_name','winner_id']].drop_duplicates().set_index('winner_id')
loser_names = df[['loser_name','loser_id']].drop_duplicates().rename(index=str, columns={"loser_name": "winner_name", "loser_id": "winner_id"}).set_index('winner_id')

In [63]:
player_unique_list = pd.concat([winner_names, loser_names]).drop_duplicates()

In [64]:
loser_names.tail()

,winner_name
winner_id,
111575,Karen Khachanov
106421,Daniil Medvedev
105967,Henri Laaksonen
104847,Adrien Bossel
134472,Antoine Bellier


In [65]:
player_unique_list[player_unique_list['winner_name'] == 'Roger Federer']

,winner_name
winner_id,
103819,Roger Federer


In [66]:
player_unique_list['elo_score'] = 1000

In [67]:
player_unique_list.head()

,winner_name,elo_score
winner_id,,
110023,Richard Coulthard,1000
109803,John Brown,1000
100257,Ross Case,1000
100105,Allan Stone,1000
109966,Warren Jacques,1000


In [68]:
df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
0,1968-580,Australian Chps.,Grass,64,G,19680119,1,110023,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1968-580,Australian Chps.,Grass,64,G,19680119,2,109803,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1968-580,Australian Chps.,Grass,64,G,19680119,3,100257,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1968-580,Australian Chps.,Grass,64,G,19680119,4,100105,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1968-580,Australian Chps.,Grass,64,G,19680119,5,109966,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
from dateutil.parser import parse

In [26]:
str(df.loc[0,'tourney_date'])

'19680119'

In [46]:
df['timestamp'] = df.loc[:,'tourney_date'].apply(lambda x: str(x))

In [47]:
df.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'winner_rank', 'winner_rank_points', 'loser_id', 'loser_seed',
       'loser_entry', 'loser_name', 'loser_hand', 'loser_ht', 'loser_ioc',
       'loser_age', 'loser_rank', 'loser_rank_points', 'score', 'best_of',
       'round', 'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon',
       'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df',
       'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved',
       'l_bpFaced', 'timestamp'],
      dtype='object')

In [54]:
df['timestamp'] = df['timestamp'].apply(lambda x: parse(x))

ValueError: Unknown string format

In [57]:
df.ix[160636]

tourney_id                    tourney_id
tourney_name                tourney_name
surface                          surface
draw_size                      draw_size
tourney_level              tourney_level
tourney_date                tourney_date
match_num                      match_num
winner_id                      winner_id
winner_seed                  winner_seed
winner_entry                winner_entry
winner_name                  winner_name
winner_hand                  winner_hand
winner_ht                      winner_ht
winner_ioc                    winner_ioc
winner_age                    winner_age
winner_rank                  winner_rank
winner_rank_points    winner_rank_points
loser_id                        loser_id
loser_seed                    loser_seed
loser_entry                  loser_entry
loser_name                    loser_name
loser_hand                    loser_hand
loser_ht                        loser_ht
loser_ioc                      loser_ioc
loser_age       